In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

c:\Users\sugay\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 指定されたプロシージャが見つかりません。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# cudaが使えるか確認
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

データの読み込み

GPUで作ったPickleのファイルは，CPUではそのままでは使えないため，工夫が必要．

https://www.kunita-gamefactory.com/post/%E3%80%90pytorch%E3%80%91gpu%E3%81%A7%E8%A8%93%E7%B7%B4%E3%81%95%E3%81%9B%E3%81%9F%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92cpu%E3%81%A7%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%82%82%E3%81%86%E3%81%A8%E3%81%97%E3%81%9F%E3%81%8A%E8%A9%B1

を真似したらうまくCPU上でもファイルを読み込むことができるようになった．

In [3]:
import io
        
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [4]:
# ファイルの相対パスを指定
file_path = '../data storage/Ising_data_L16.pkl'

if torch.cuda.is_available():
    with open(file_path, 'rb') as file:
        loaded_data = pickle.load(file)
else:
    with open(file_path, 'rb') as file:
        loaded_data = CPU_Unpickler(file).load()
        
# 読み込んだデータを個々の変数に分割
spin_data, label_data = loaded_data

In [5]:
spin_data[17332]

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [-1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1

磁化をすべて正に変える

In [6]:
# 磁化を計算
def magnetization(state):
    return np.mean(state)

for i in range(len(spin_data)):
    mag = magnetization(spin_data[i])
    if mag > 0:
        spin_data[i] *= -1

In [7]:
spin_data[17332]

array([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.,
        -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
        -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1.,  1.,  1., -1., -1

In [8]:
# データのリストをNumPy配列に変換
spin_data_np = np.array(spin_data)
label_data_np = np.array(label_data)

# NumPy配列をPyTorchテンソルに変換
spin_data_tensor = torch.from_numpy(spin_data_np)
label_data_tensor = torch.from_numpy(label_data_np)

In [9]:
spin_data_tensor = spin_data_tensor.unsqueeze(1)
spin_data_tensor.shape

torch.Size([40000, 1, 16, 16])

In [10]:
# サンプルデータを訓練用とテスト用に分割(5:5)
spin_train, spin_test, label_train, label_test = train_test_split(spin_data_tensor, label_data_tensor, test_size=0.5)

In [11]:
# PyTorchのテンソルに変換
spin_train = torch.tensor(spin_train, dtype=torch.float32)
spin_test = torch.tensor(spin_test, dtype=torch.float32)
label_train = torch.tensor(label_train, dtype=torch.float32)  
label_test = torch.tensor(label_test, dtype=torch.float32)    

C:\Users\sugay\AppData\Local\Temp\ipykernel_10428\168836570.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spin_train = torch.tensor(spin_train, dtype=torch.float32)
C:\Users\sugay\AppData\Local\Temp\ipykernel_10428\168836570.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spin_test = torch.tensor(spin_test, dtype=torch.float32)
C:\Users\sugay\AppData\Local\Temp\ipykernel_10428\168836570.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train, dtype=torch.float32)
C:\Users\sugay\AppData\Local\Te

正解データはone-hot表現にする必要がある

In [12]:
# テンソルを新しいテンソルに変換する関数を定義
def to_one_hot(data, num_classes=20):
    # one-hotベクトルの初期化
    one_hot = torch.zeros(len(data), num_classes)
    # 各要素を20次元のone-hotベクトルに変換
    for i, val in enumerate(data):
        index = int(torch.round((val - 0.05) / 0.05))
        one_hot[i, index] = 1.0
    
    return one_hot

# label_train,temp_testをone-hotベクトルに変換
one_hot_label_train = to_one_hot(label_train, num_classes=20)
one_hot_label_test = to_one_hot(label_test, num_classes=20)

In [13]:
# データセットの作成
train_dataset = TensorDataset(spin_train, one_hot_label_train)
test_dataset = TensorDataset(spin_test, one_hot_label_test)

In [14]:
# 前処理を定義
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
])

# データセットに前処理を適用
transformed_train_dataset = [(transform(tensor_sample), label) for tensor_sample, label in train_dataset]
transformed_test_dataset = [(transform(tensor_sample), label) for tensor_sample, label in test_dataset]

In [15]:
# DataLoaderの設定
train_loader = DataLoader(transformed_train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(transformed_test_dataset, batch_size=16)

In [16]:
# # 前処理なしver
# # DataLoaderの設定（バッチサイズ50）
# train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=50)

モデルを定義

In [17]:
class CNN(nn.Module):
    def __init__(self,output_size):
        super(CNN, self).__init__()
        
        # 畳み込み層
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=2, stride=2, bias=False)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=2, stride=2, bias=False)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=3, kernel_size=4, stride=4, bias=False)
        
        # 畳み込みの部分
        self.conv = nn.Sequential(
            self.conv1,   
            nn.ReLU(inplace=True),
            self.conv2,   
            nn.ReLU(inplace=True),
            self.conv3,   
            nn.ReLU(inplace=True)
        )
        # 全結合の部分
        self.fc = nn.Sequential(
            nn.Linear(3, output_size, bias=False),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


モデルのインスタンス化

In [18]:
input_size = 16*16
output_size = 20

model = CNN(output_size)
model.to(device)
# モデルの概要表示
print(model)

CNN(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(2, 2), bias=False)
  (conv2): Conv2d(64, 32, kernel_size=(2, 2), stride=(2, 2), bias=False)
  (conv3): Conv2d(32, 3, kernel_size=(4, 4), stride=(4, 4), bias=False)
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(2, 2), stride=(2, 2), bias=False)
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 32, kernel_size=(2, 2), stride=(2, 2), bias=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(32, 3, kernel_size=(4, 4), stride=(4, 4), bias=False)
    (5): ReLU(inplace=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=3, out_features=20, bias=False)
    (1): Softmax(dim=1)
  )
)


損失関数と最適化アルゴリズムの設定

In [19]:
criterion = nn.CrossEntropyLoss()   # クロスエントロピー誤差
optimizer = optim.Adam(model.parameters(), lr=0.0001)     # Adam,L2正則化{, weight_decay=5e-4}

学習の実行

In [20]:
num_epochs = 100
train_losses = []
train_accs = []
test_losses = []
test_accs = []
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        running_loss += loss.item()
        pred = torch.argmax(output, dim=1)      
        targets = torch.argmax(targets, dim=1) 
        running_acc += torch.mean(pred.eq(targets).float().cpu()) 
        optimizer.step()
    running_loss /= len(train_loader)   
    running_acc /= len(train_loader)    
    train_losses.append(running_loss)
    train_accs.append(running_acc)
    #
    #   test loop
    #
    test_running_loss = 0.0
    test_running_acc = 0.0
    for test_inputs, test_targets in test_loader:
        test_inputs = test_inputs.to(device)
        test_targets = test_targets.to(device)
        test_output = model(test_inputs)
        test_loss = criterion(test_output, test_targets)
        test_running_loss += test_loss.item()
        test_pred = torch.argmax(test_output, dim=1)      
        test_targets = torch.argmax(test_targets, dim=1)  
        test_running_acc += torch.mean(test_pred.eq(test_targets).float().cpu()) 
    test_running_loss /= len(test_loader)   
    test_running_acc /= len(test_loader)    
    test_losses.append(test_running_loss)
    test_accs.append(test_running_acc)
        
    print("epoch: {}, loss: {}, acc: {}, test loss: {}, test acc: {}".format(epoch, running_loss, running_acc, test_running_loss, test_running_acc))

epoch: 0, loss: 2.9618075801849364, acc: 0.0957999974489212, test loss: 2.948749828720093, test acc: 0.1050499975681305
epoch: 1, loss: 2.941056983757019, acc: 0.11065000295639038, test loss: 2.938610462188721, test acc: 0.11010000109672546
epoch: 2, loss: 2.930813435745239, acc: 0.12035000324249268, test loss: 2.929525666618347, test acc: 0.12274999916553497
epoch: 3, loss: 2.922883764076233, acc: 0.1253499984741211, test loss: 2.9232952276229858, test acc: 0.12449999898672104
epoch: 4, loss: 2.9173422939300537, acc: 0.1264999955892563, test loss: 2.918678924369812, test acc: 0.12070000171661377
epoch: 5, loss: 2.9127872468948364, acc: 0.1264999955892563, test loss: 2.9137081438064576, test acc: 0.12309999763965607
epoch: 6, loss: 2.9074185703277586, acc: 0.14614999294281006, test loss: 2.908987877082825, test acc: 0.1569499969482422
epoch: 7, loss: 2.9013726001739504, acc: 0.15905000269412994, test loss: 2.9014684387207033, test acc: 0.15690000355243683


KeyboardInterrupt: 

モデルの保存

In [ ]:
Ising_size = int(np.sqrt(input_size))
hidden_size = 3
class_name = type(model).__name__

file_path = f'../data storage/prm_data_L{Ising_size}_{class_name}_Nh{hidden_size}.pth'

# torch.save(model.state_dict(), file_path)